# I. Deep Learning: Classification

### 1. La Classification consiste à :
- charger les données
- les transmettre à travers le modèle
- générer une sortie
- calculer la perte, 
- prendre des gradients par rapport aux poids et mettre à jour le modèle. 
#### Cependant, la forme précise des cibles, le paramétrage de la couche de sortie et le choix de la fonction de perte s'adapteront au réglage de la classification.

### 2. Dans cette première partie, nous nous concentrons sur les problèmes de classification où on va plutôt se concentrer sur la question suivante: quelle catégorie appartient cet objet/image ?.
#### Exemple de questions:
- Cet e-mail appartient-il au dossier spam ou à la boîte de réception ?
- Ce client est-il plus susceptible de souscrire ou non à un service d'abonnement ?
- Cette image représente-t-elle un âne, un chien, un chat ou un coq ?
- Quel film Aston est-il le plus susceptible de regarder ensuite ?
- Quelle section du livre allez-vous lire ensuite?


- Pour se mouiller les pieds, commençons par un simple problème de classification d'images. Ici, chaque entrée consiste en une image $2\times2$ en niveaux de gris. Nous pouvons représenter chaque valeur de pixel avec un seul scalaire, nous donnant quatre caractéristiques $x_1, x_2, x_3, x_4$. De plus, supposons que chaque image appartient à l'une des catégories "chat", "poulet" et "chien".

- Ensuite, nous devons choisir comment représenter les étiquettes. Nous avons deux choix évidents. L'impulsion la plus naturelle serait peut-être de choisir $y \in \{1, 2, 3\}$, où les nombres entiers représentent $\{\text{dog}, \text{cat}, \text{chicken}\}$ respectivement. C'est un excellent moyen de stocker ces informations sur un ordinateur. Si les catégories avaient un ordre naturel entre elles, disons si nous essayions de prédire $\{\text{baby}, \text{toddler}, \text{adolescent}, \text{young adult}, \text{adult}, \text{geriatric}\}$, alors il pourrait même être judicieux de le présenter comme un problème de régression ordinale [ordinal regression](https://en.wikipedia.org/wiki/Ordinal_regression) et de conserver les étiquettes dans ce format. Voir Moon et al. ( 2010 ) :citet:`Moon.Smola.Chang.ea.2010` pour un aperçu des différents types de fonctions de perte de classement et Beutel et al. ( 2014 ) :citet:`Beutel.Murray.Faloutsos.ea.2014` pour une approche bayésienne qui aborde les réponses avec plus d'un mode.

- En général, les problèmes de classification ne s'accompagnent pas d'ordres naturels entre les classes. Heureusement, les statisticiens ont depuis longtemps inventé un moyen simple de représenter les données catégorielles : le codage à chaud . Un encodage $\{\text{one-hot}\}$ est un vecteur avec autant de composants que nous avons de catégories. Le composant correspondant à la catégorie d'une instance particulière est défini sur 1 et tous les autres composants sont définis sur 0. Dans notre cas, une étiquette $y$ serait un vecteur tridimensionnel, avec $(1, 0, 0)$ correspondant à « chat », $(0, 1, 0)$ au "poulet", et $(0, 0, 1)$ au chien":

$$y \in \{(1, 0, 0), (0, 1, 0), (0, 0, 1)\}.$$

### 3. Modèle Linéaire.
- Afin d'estimer les probabilités conditionnelles associées à toutes les classes possibles, nous avons besoin d'un modèle à sorties multiples, une par classe. Pour aborder la classification avec des modèles linéaires, nous aurons besoin d'autant de fonctions affines que nous avons de sorties. À proprement parler, nous n'en avons besoin que d'un de moins, puisque la dernière catégorie doit être la différence entre $1$ et la somme des autres catégories mais pour des raisons de symétrie nous utilisons une paramétrisation légèrement redondante. Chaque sortie correspond à sa propre fonction affine. Dans notre cas, puisque nous avons 4 caractéristiques et 3 catégories de sortie possibles, nous avons besoin de 12 scalaires pour représenter les poids ($w$ avec des indices), et 3 scalaires pour représenter les biais ($b$ avec des indices). Cela donne :

$$
\begin{aligned}
o_1 &= x_1 w_{11} + x_2 w_{12} + x_3 w_{13} + x_4 w_{14} + b_1,\\
o_2 &= x_1 w_{21} + x_2 w_{22} + x_3 w_{23} + x_4 w_{24} + b_2,\\
o_3 &= x_1 w_{31} + x_2 w_{32} + x_3 w_{33} + x_4 w_{34} + b_3.
\end{aligned}
$$

- Tout comme dans la régression linéaire, nous utilisons un réseau de neurones à une seule couche. Et depuis le calcul de chaque sortie,  $o_1, o_2$, et $o_3$, dépend de toutes les entrées $x_1$, $x_2$, $x_3$, et $x_4$, la couche de sortie peut également être décrite comme une couche entièrement connectée.

- Pour une notation plus concise, nous utilisons des vecteurs et des matrices $\mathbf{o} = \mathbf{W} \mathbf{x} + \mathbf{b}$: est beaucoup mieux adapté aux mathématiques et au code. Notez que nous avons rassemblé tous nos poids dans une $3 \times 4$ matrice et tous les biais $\mathbf{b} \in \mathbb{R}^3$ dans un vecteur.

### 4. Le Softmax
- En supposant une fonction de perte appropriée, nous pourrions essayer, directement, de minimiser la différence entre $\mathbf{o}$ et les étiquettes $\mathbf{y}$. S'il s'avère que le traitement de la classification comme un problème de régression à valeur vectorielle fonctionne étonnamment bien, il manque néanmoins les éléments suivants :

    * Il n'y a aucune garantie que les sorties $o_i$ somme jusqu'à $1$ dans la façon dont nous nous attendons à ce que les probabilités se comportent.

    * Il n'y a aucune garantie que les sorties $o_i$ sont même non négatifs, même si leurs sorties totalisent $1$, ou qu'ils ne dépassent pas $1$.

- Les deux aspects rendent le problème d'estimation difficile à résoudre et la solution très fragile aux valeurs aberrantes. Par exemple, si nous supposons qu'il existe une dépendance linéaire positive entre le nombre de chambres et la probabilité que quelqu'un achète une maison, la probabilité pourrait dépasser $1$ quand il s'agit d'acheter un manoir! En tant que tel, nous avons besoin d'un mécanisme pour "écraser" les sorties.

- Il existe de nombreuses façons d'atteindre cet objectif. Par exemple, nous pourrions supposer que les sorties $o_i$ sont des versions corrompues de $y$, où la corruption se produit au moyen de l'ajout de bruit $\mathbf{\epsilon}$ tirée d'une distribution normale. Autrement dit, $\mathbf{y} = \mathbf{o} + \mathbf{\epsilon}$, où $\epsilon_i \sim \mathcal{N}(0, \sigma^2)$. C'est ce qu'on appelle le modèle probit [probit model](https://en.wikipedia.org/wiki/Probit_model), introduit pour la première fois par Fechner ( 1860 ) :citet:`Fechner.1860`. Bien qu'attirant, cela ne fonctionne pas aussi bien ou conduit à un problème d'optimisation particulièrement agréable, par rapport au softmax.

- Une autre façon d'atteindre cet objectif (et d'assurer la non-négativité) qui consiste à utiliser une fonction exponentielle $P(y = i) \propto \exp o_i$. Cela satisfait en effet l'exigence selon laquelle la probabilité de classe conditionnelle augmente avec l'augmentation $o_i$, elle est monotone et toutes les probabilités sont non négatives. Nous pouvons ensuite transformer ces valeurs pour qu'elles s'additionnent à $1$ en divisant chacun par leur somme. Ce processus est appelé *normalisation* . L'assemblage de ces deux éléments nous donne la fonction *softmax*:

$$\hat{\mathbf{y}} = \mathrm{softmax}(\mathbf{o}) \quad \text{where}\quad \hat{y}_i = \frac{\exp(o_i)}{\sum_j \exp(o_j)}.$$

- Notez que la plus grande coordonnée de $\mathbf{o}$ correspond à la classe la plus probable selon $\hat{\mathbf{y}}$. De plus, comme l'opération softmax préserve l'ordre parmi ses arguments, nous n'avons pas besoin de calculer le softmax pour déterminer à quelle classe a été attribuée la probabilité la plus élevée.

$$ \operatorname*{argmax}_j \hat y_j = \operatorname*{argmax}_j o_j.$$

- L'idée d'un softmax remonte à Gibbs, qui a adapté des idées de la physique ( Gibbs, 1902 ) :cite:`Gibbs.1902`. Datant encore plus loin, Boltzmann, le père de la thermodynamique moderne, a utilisé cette astuce pour modéliser une distribution sur les états d'énergie dans les molécules de gaz. En particulier, il a découvert que la prévalence d'un état d'énergie dans un ensemble thermodynamique, comme les molécules d'un gaz, est proportionnelle à $\exp(-E/kT)$. Ici, $E$ est l'énergie d'un état, $T$ est la température, et $k$ est la constante de Boltzmann. Lorsque les statisticiens parlent d'augmenter ou de diminuer la « température » d'un système statistique, ils se réfèrent à l'évolution $T$ afin de favoriser des états d'énergie plus ou moins élevés. Selon l'idée de Gibbs, l'énergie équivaut à l'erreur. Les modèles basés sur l'énergie ( Ranzato et al. , 2007 ) utilisent ce point de vue pour décrire les problèmes d'apprentissage en profondeur.

### 4. Vectorisation

- Pour améliorer l'efficacité des calculs, nous vectorisons les calculs en mini-lots de données. Supposons qu'on nous donne un mini-lot $\mathbf{X} \in \mathbb{R}^{n \times d}$ de $n$ caractéristiques avec dimensionnalité (nombre d'entrées) $d$. De plus, supposons que nous ayons $q$ catégories dans la sortie. Alors les poids satisfont $\mathbf{W} \in \mathbb{R}^{d \times q}$ et le biais satisfait $\mathbf{b} \in \mathbb{R}^{1\times q}$.

$$ \begin{aligned} \mathbf{O} &= \mathbf{X} \mathbf{W} + \mathbf{b}, \\ 
\hat{\mathbf{Y}} & = \mathrm{softmax}(\mathbf{O}). \end{aligned} $$

- Cela accélère l'opération dominante dans un produit matrice-matrice $\mathbf{X} \mathbf{W}$. De plus, étant donné que chaque ligne de $\mathbf{X}$ représente un exemple de données, l'opération softmax elle-même peut être calculée par *ligne*: pour chaque ligne de $\mathbf{O}$, exponentiez toutes les entrées, puis normalisez-les par la somme. Notez, cependant, que des précautions doivent être prises pour éviter d'exposer et de prendre des logarithmes de grands nombres, car cela peut provoquer un débordement ou un sous-dépassement numérique. Les frameworks d'apprentissage en profondeur s'en chargent automatiquement.

### 4. Fonction de Perte
- Maintenant que nous avons un mappage à partir des fonctionnalités $\mathbf{x}$ aux probabilités $\mathbf{\hat{y}}$, nous avons besoin d'un moyen d'optimiser la précision de cette cartographie. Nous nous appuierons sur l'estimation du maximum de vraisemblance.

#### 4.1. Log-Vraisemblance (Log-Likelihood)
- La fonction softmax nous donne un vecteur $\hat{\mathbf{y}}$, que nous pouvons interpréter comme des probabilités conditionnelles (estimées) de chaque classe, compte tenu de toute entrée $\mathbf{x}$, tel que $\hat{y}_1$ = $P(y=\text{cat} \mid \mathbf{x})$. Dans ce qui suit, nous supposons que pour un jeu de données avec des fonctionnalités $\mathbf{X}$ les étiquettes $\mathbf{Y}$ sont représentés à l'aide d'un vecteur d'étiquette de codage à chaud (one-hot). Nous pouvons comparer les estimations avec la réalité en vérifiant la probabilité des classes réelles selon notre modèle, compte tenu des caractéristiques :

$$
P(\mathbf{Y} \mid \mathbf{X}) = \prod_{i=1}^n P(\mathbf{y}^{(i)} \mid \mathbf{x}^{(i)}).
$$

- Nous sommes autorisés à utiliser la factorisation puisque nous supposons que chaque étiquette est tirée indépendamment de sa distribution respective . Étant donné que la maximisation du produit des termes est maladroite, nous prenons le logarithme négatif pour obtenir le problème équivalent de la minimisation de la log-vraisemblance négative :

$$
-\log P(\mathbf{Y} \mid \mathbf{X}) = \sum_{i=1}^n -\log P(\mathbf{y}^{(i)} \mid \mathbf{x}^{(i)})
= \sum_{i=1}^n l(\mathbf{y}^{(i)}, \hat{\mathbf{y}}^{(i)}),
$$

- où pour toute paire d'étiquettes $\mathbf{y}$ et prédiction du modèle $\hat{\mathbf{y}}$ pour $q$ classes, la fonction de perte $l$ est:

$$ l(\mathbf{y}, \hat{\mathbf{y}}) = - \sum_{j=1}^q y_j \log \hat{y}_j. $$

- Pour des raisons expliquées plus loin, la fonction de perte est communément appelée perte d'entropie croisée. Depuis $\mathbf{y}$ est un vecteur one-hot de longueur $q$, la somme sur toutes ses coordonnées $j$ disparaît pour tous les termes sauf un. A noter que la perte $l(\mathbf{y}, \hat{\mathbf{y}})$ est délimité par le bas par $0$ chaque fois que $\hat{y}$ est un vecteur de probabilité : aucune entrée n'est plus grande que $1$, donc leur logarithme négatif ne peut pas être inférieur à $0$; $l(\mathbf{y}, \hat{\mathbf{y}}) = 0$ seulement si nous prédisons l'étiquette réelle avec certitude. Cela ne peut jamais arriver pour un réglage fini des poids car prendre une sortie softmax vers $1$ nécessite de prendre l'entrée correspondante $o_i$ à l'infini (ou toutes les autres sorties $o_j$ pour $j \neq i$ moins l'infini). Même si notre modèle pouvait attribuer une probabilité de sortie de $0$, toute erreur commise lors de l'attribution d'un niveau de confiance aussi élevé entraînerait une perte infinie ($-\log 0 = \infty$).

#### 4.2. Softmax et perte d'entropie croisée

- Étant donné que la fonction softmax et la perte d'entropie croisée correspondante sont si courantes, il est utile de comprendre un peu mieux comment elles sont calculées. En branchant la fonction: $$\hat{\mathbf{y}} = \mathrm{softmax}(\mathbf{o}) \quad \text{where}\quad \hat{y}_i = \frac{\exp(o_i)}{\sum_j \exp(o_j)}.$$ dans la définition de la perte en $$ l(\mathbf{y}, \hat{\mathbf{y}}) = - \sum_{j=1}^q y_j \log \hat{y}_j. $$ et en utilisant la définition du softmax on obtient:

$$
\begin{aligned}
l(\mathbf{y}, \hat{\mathbf{y}}) &=  - \sum_{j=1}^q y_j \log \frac{\exp(o_j)}{\sum_{k=1}^q \exp(o_k)} \\
&= \sum_{j=1}^q y_j \log \sum_{k=1}^q \exp(o_k) - \sum_{j=1}^q y_j o_j \\
&= \log \sum_{k=1}^q \exp(o_k) - \sum_{j=1}^q y_j o_j.
\end{aligned}
$$

- Pour comprendre un peu mieux ce qui se passe, considérons la dérivée par rapport à tout logit $o_j$. On a:

$$
\partial_{o_j} l(\mathbf{y}, \hat{\mathbf{y}}) = \frac{\exp(o_j)}{\sum_{k=1}^q \exp(o_k)} - y_j = \mathrm{softmax}(\mathbf{o})_j - y_j.
$$

- En d'autres termes, la dérivée est la différence entre la probabilité attribuée par notre modèle, telle qu'exprimée par l'opération softmax, et ce qui s'est réellement passé, tel qu'exprimé par les éléments du vecteur d'étiquette à chaud. En ce sens, cela ressemble beaucoup au problème de la régression, où le gradient est la différence entre l'observation $y$ et la valeur estimée $\hat{y}$. Ce n'est pas une coïncidence. Dans tout modèle de famille exponentielle, les gradients de la log-vraisemblance sont précisément donnés par ce terme. Ce fait facilite le calcul des gradients dans la pratique.

- Considérons maintenant le cas où nous observons non seulement un seul résultat, mais une distribution complète des résultats. On peut utiliser la même représentation que précédemment pour l'étiquette $\mathbf{y}$. La seule différence est que plutôt qu'un vecteur contenant uniquement des entrées binaires, disons $(0, 0, 1)$, nous avons maintenant un vecteur de probabilité générique, disons $(0.1, 0.2, 0.7)$. Les mathématiques que nous avons utilisées précédemment pour définir la perte $l$: $$ l(\mathbf{y}, \hat{\mathbf{y}}) = - \sum_{j=1}^q y_j \log \hat{y}_j. $$ fonctionne toujours bien, juste que l'interprétation est légèrement plus générale. C'est la valeur attendue de la perte pour une distribution sur les étiquettes. Cette perte est appelée perte d'entropie croisée et c'est l'une des pertes les plus couramment utilisées pour les problèmes de classification. Nous pouvons démystifier le nom en introduisant simplement les bases de la théorie de l'information. En un mot, il mesure le nombre de bits pour coder ce que nous voyons $y$ par rapport à ce que nous prévoyons $\hat{\mathbf{y}}$ qui devrait arriver. Pour plus de détails sur la théorie de l'information, voir ( Cover et Thomas, 1999 ) ou ( MacKay et Mac Kay, 2003 ) .

# II. Classification d'images (MNIST) avec l'API Keras

- L'un des ensembles de données largement utilisés pour la classification des images est l' ensemble de données MNIST ( LeCun et al. , 1998 ) [MNIST dataset](https://en.wikipedia.org/wiki/MNIST_database)  de chiffres manuscrits. Au moment de sa sortie dans les années 1990, il posait un formidable défi à la plupart des algorithmes d'apprentissage automatique, composé de 60 000 images de résolution $28 \times 28$ pixels en pixels (plus un ensemble de données de test de 10 000 images).

- Pendant plus d'une décennie, le MNIST a servi de point de référence pour comparer les algorithmes d'apprentissage automatique. Bien qu'il ait bien fonctionné en tant qu'ensemble de données de référence, même les modèles simples selon les normes actuelles atteignent une précision de classification supérieure à 95 %, ce qui le rend inadapté pour faire la distinction entre les modèles les plus forts et les plus faibles. Plus encore, l'ensemble de données permet des niveaux de précision très élevés, rarement observés dans de nombreux problèmes de classification. Ce développement algorithmique biaisé vers des familles spécifiques d'algorithmes qui peuvent tirer parti d'ensembles de données propres, tels que les méthodes d'ensembles actifs et les algorithmes d'ensembles actifs de recherche de limites. Aujourd'hui, le MNIST sert davantage de vérification de la santé mentale que de référence. ImageNet ( Deng et al. , 2009 )pose un défi beaucoup plus pertinent. Malheureusement, ImageNet est trop volumineux pour la plupart des exemples et illustrations de ce livre, car il faudrait trop de temps pour s'entraîner à rendre les exemples interactifs. Au lieu de cela, nous concentrerons notre discussion dans les sections à venir sur l'ensemble de données Fashion-MNIST qualitativement similaire, mais beaucoup plus petit ( Xiao et al. , 2017 ) , qui a été publié en 2017. Il contient des images de 10 catégories de vêtements à résolution $28 \times 28$ en pixels.

## 1. Chargement du jeu de données¶

- Comme il s'agit d'un ensemble de données si fréquemment utilisé, tous les principaux frameworks en fournissent des versions prétraitées. Nous pouvons télécharger et lire l'ensemble de données Fashion-MNIST en mémoire à l'aide de fonctions de cadre intégrées.

### 1.1. Importer les packages suivants:

In [2]:
%pip install d2l==1.0.0a1.post0
import time
import tensorflow as tf
from d2l import tensorflow as d2l
d2l.use_svg_display()

  Using cached d2l-1.0.0a1.post0-py3-none-any.whl (93 kB)
  Using cached gym-0.26.2.tar.gz (721 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Using cached gym_notices-0.0.8-py3-none-any.whl (3.0 kB)


  Created wheel for gym: filename=gym-0.26.2-py3-none-any.whl size=827636 sha256=df36d502800788f8067867572d72425f7b495cfbce7e301ccc70f5484b4706e6
  Stored in directory: /home/vincent/.cache/pip/wheels/af/2b/30/5e78b8b9599f2a2286a582b8da80594f654bf0e18d825a4405
Successfully built gym
Note: you may need to restart the kernel to use updated packages.


### 1.2. Chargement de la base MNIST depuis Keras

- La première étape consiste à charger les données MNIST selon la documentation de keras https://keras.io/datasets/

In [4]:
class FashionMNIST(d2l.DataModule):  #@save
    def __init__(self, batch_size=64, resize=(28, 28)):
        super().__init__()
        self.save_hyperparameters()
        self.train, self.val = tf.keras.datasets.fashion_mnist.load_data()

- Fashion-MNIST se compose d'images de 10 catégories, chacune représentée par 6 000 images dans l'ensemble de données d'entraînement et par 1 000 dans l'ensemble de données de test. Un jeu de données de test est utilisé pour évaluer les performances du modèle (il ne doit pas être utilisé pour la formation). Par conséquent, l'ensemble d'apprentissage et l'ensemble de test contiennent respectivement 60 000 et 10 000 images.

In [5]:
data = FashionMNIST(resize=(32, 32))
len(data.train[0]), len(data.val[0])

(60000, 10000)

- Les images sont en niveaux de gris et mises à l'échelle pour $32 \times 32$ pixels en résolution ci-dessus. Ceci est similaire à l'ensemble de données MNIST original qui se composait d'images (binaires) en noir et blanc. Notez, cependant, que la plupart des données d'image modernes qui ont 3 canaux (rouge, vert, bleu) et des images hyperspectrales qui peuvent avoir plus de 100 canaux (le capteur HyMap a 126 canaux). Par convention, nous stockons l'image en tant que tenseur $c \times h \times w$, où $c$ est le nombre de canaux de couleur, $h$ est la hauteur et $w$ est la largeur.

In [6]:
data.train[0][0].shape

(28, 28)

- Les catégories de Fashion-MNIST ont des noms compréhensibles par l'homme. La fonction pratique suivante effectue la conversion entre les étiquettes numériques et leurs noms.

In [7]:
@d2l.add_to_class(FashionMNIST)  #@save
def text_labels(self, indices):
    """Return text labels."""
    labels = ['t-shirt', 'trouser', 'pullover', 'dress', 'coat', 'sandal', 'shirt', 'sneaker', 'bag', 'ankle boot']
    return [labels[int(i)] for i in indices]

### 1.3. Lecture d'un Minibatch

- Pour nous faciliter la vie lors de la lecture des ensembles d'entraînement et de test, nous utilisons l'itérateur de données intégré plutôt que d'en créer un à partir de zéro. Rappelons qu'à chaque itération, un itérateur de données lit un mini-lot de données de taille batch_size. Nous mélangeons également au hasard les exemples pour l'itérateur de données d'apprentissage.

In [ ]:
@d2l.add_to_class(FashionMNIST)  #@save
def get_dataloader(self, train):
    data = self.train if train else self.val
    process = lambda X, y: (tf.expand_dims(X, axis=3) / 255,
                            tf.cast(y, dtype='int32'))
    resize_fn = lambda X, y: (tf.image.resize_with_pad(X, *self.resize), y)
    shuffle_buf = len(data[0]) if train else 1
    return tf.data.Dataset.from_tensor_slices(process(*data)).batch(
        self.batch_size).map(resize_fn).shuffle(shuffle_buf)

- Pour voir comment cela fonctionne, chargeons un mini-lot d'images en appelant la *train_dataloader* méthode nouvellement ajoutée. Il contient 64 images.

In [ ]:
X, y = next(iter(data.train_dataloader()))
print(X.shape, X.dtype, y.shape, y.dtype)

- Regardons le temps qu'il faut pour lire les images. Même s'il s'agit d'un chargeur intégré, il n'est pas extrêmement rapide. Néanmoins, cela est suffisant car le traitement des images avec un réseau profond prend un peu plus de temps. Par conséquent, il est suffisant que la formation d'un réseau ne soit pas contrainte par les E/S.

In [ ]:
tic = time.time()
for X, y in data.train_dataloader():
    continue
f'{time.time() - tic:.2f} sec'

### 1.3. Visualisation

- Nous utiliserons assez fréquemment l'ensemble de données Fashion-MNIST. Une fonction de commodité show_imagespermet de visualiser les images et les étiquettes associées. Les détails de sa mise en œuvre sont reportés en annexe.

In [ ]:
def show_images(imgs, num_rows, num_cols, titles=None, scale=1.5):  #@save
    """Plot a list of images."""
    raise NotImplementedError

- Faisons-en bon usage. En général, c'est une bonne idée de visualiser et d'inspecter les données sur lesquelles vous vous entraînez. Les humains sont très doués pour repérer les aspects inhabituels et, à ce titre, la visualisation sert de protection supplémentaire contre les erreurs et les erreurs dans la conception des expériences. Voici les images et leurs étiquettes correspondantes (en texte) pour les premiers exemples de l'ensemble de données d'entraînement.

In [ ]:
"""@d2l.add_to_class(FashionMNIST)  #@save
def visualize(self, batch, nrows=1, ncols=8, labels=[]):
    X, y = batch
    if not labels:
        labels = self.text_labels(y)
    d2l.show_images(tf.squeeze(X), nrows, ncols, titles=labels)

batch = next(iter(data.val_dataloader()))
data.visualize(batch)"""

### 1.4. Conclusion

- Nous avons maintenant un ensemble de données légèrement plus réaliste à utiliser pour la classification. Fashion-MNIST est un ensemble de données de classification de vêtements composé d'images représentant 10 catégories. Nous utiliserons cet ensemble de données dans les sections et chapitres suivants pour évaluer diverses conceptions de réseau, d'un modèle linéaire simple à des réseaux résiduels avancés. Comme nous le faisons couramment avec les images, nous les lisons comme un tenseur de forme (taille du lot, nombre de canaux, hauteur, largeur). Pour l'instant, nous n'avons qu'un seul canal car les images sont en niveaux de gris (la visualisation ci-dessus utilise une fausse palette de couleurs pour une meilleure visibilité).

- Enfin, les itérateurs de données sont un élément clé pour des performances efficaces. Par exemple, nous pourrions utiliser des GPU pour une décompression d'image efficace, un transcodage vidéo ou d'autres prétraitements. Dans la mesure du possible, vous devez vous fier à des itérateurs de données bien implémentés qui exploitent le calcul haute performance pour éviter de ralentir votre boucle de formation.

## 2. Le Modèle de classification de base 

### 2.1.  Le modèle de classification de base

- Nous définissons la Classifierclasse ci-dessous. Dans le validation_step, nous rapportons à la fois la valeur de perte et la précision de la classification sur un lot de validation. Nous dessinons une mise à jour pour chaque num_val_batches lot. Cela a l'avantage de générer la perte moyenne et la précision sur l'ensemble des données de validation. Ces nombres moyens ne sont pas exactement corrects si le dernier lot contient moins d'exemples, mais nous ignorons cette différence mineure pour garder le code simple.

In [ ]:
class Classifier(d2l.Module):  #@save
    def validation_step(self, batch):
        Y_hat = self(*batch[:-1])
        self.plot('loss', self.loss(Y_hat, batch[-1]), train=False)
        self.plot('acc', self.accuracy(Y_hat, batch[-1]), train=False)

- Par défaut, nous utilisons un optimiseur de descente de gradient stochastique (SGD), fonctionnant sur des mini-lots, tout comme nous l'avons fait dans le cadre de la régression linéaire.

In [ ]:
@d2l.add_to_class(d2l.Module)  #@save
def configure_optimizers(self):
    return tf.keras.optimizers.SGD(self.lr)

### 2.2. Précision

- Compte tenu de la distribution de probabilité prédite `y_hat`, nous choisissons généralement la classe avec la probabilité prédite la plus élevée chaque fois que nous devons produire une prédiction dure. En effet, de nombreuses applications nécessitent que l'on fasse un choix. Par exemple, Gmail doit classer un e-mail dans "Primaire", "Social", "Mises à jour", "Forums" ou "Spam". Il peut estimer les probabilités en interne, mais en fin de compte, il doit en choisir une parmi les classes.

- Lorsque les prédictions sont cohérentes avec la classe d'étiquettes `y`, elles sont correctes. La précision de la classification est la fraction de toutes les prédictions qui sont correctes. Bien qu'il puisse être difficile d'optimiser directement la précision (elle n'est pas différentiable), c'est souvent la mesure de performance qui nous importe le plus. C'est souvent la quantité pertinente dans les benchmarks. En tant que tel, nous le signalerons presque toujours lors de la formation des classificateurs.

- La précision est calculée comme suit. Premièrement, si `y_hat` est une matrice, nous supposons que la deuxième dimension stocke les scores de prédiction pour chaque classe. Nous utilisons `argmax` pour obtenir la classe prédite par l'indice pour la plus grande entrée de chaque ligne. Ensuite, nous [**comparons la classe prédite avec la vérité terrain `y` élément par élément.**] . Étant donné que l'opérateur d'égalité `==` est sensible aux types de données, nous convertissons `y_hat` le type de données de pour qu'il corresponde à celui de `y`. Le résultat est un tenseur contenant les entrées 0 (faux) et 1 (vrai). La somme donne le nombre de prédictions correctes.

In [ ]:
@d2l.add_to_class(Classifier)  #@save
def accuracy(self, Y_hat, Y, averaged=True):
    """Compute the number of correct predictions."""
    Y_hat = tf.reshape(Y_hat, (-1, Y_hat.shape[-1]))
    preds = tf.cast(tf.argmax(Y_hat, axis=1), Y.dtype)
    compare = tf.cast(preds == tf.reshape(Y, -1), tf.float32)
    return tf.reduce_mean(compare) if averaged else compare

### 2.3. Conclusion

- La classification est un problème suffisamment courant pour justifier ses propres fonctions de commodité. La précision du classifieur est d'une importance capitale dans la classification. Notez que même si nous nous soucions souvent principalement de la précision, nous formons des classifieurs pour optimiser une variété d'autres objectifs pour des raisons statistiques et informatiques. Cependant, quelle que soit la fonction de perte qui a été minimisée pendant la formation, il est utile de disposer d'une méthode pratique pour évaluer empiriquement la précision de notre classificateur.